In [1]:
import io
from contextlib import redirect_stdout

from pyspark.sql import functions as F, SparkSession

In [3]:
from pyspark.sql import functions as F, SparkSession

spark = SparkSession.builder.getOrCreate()
check = spark.range(100)
check = check.withColumn("user_id", F.lit(1))
check = check.withColumn("sales_amount", F.lit(1))
check = check.withColumn("date", F.lit(1))
#check.write.parquet("example_data", mode="overwrite")

In [95]:
df = spark.read.load("example_data")
dd = df.groupBy("user_id").agg(F.sum("sales_amount").alias("sales"), F.countDistinct("date").alias("date_distinct"))
dd = dd.withColumn("distinct1", F.lit(1) + F.col("date_distinct"))
dd = dd.agg(F.sum("sales").alias("sales"), F.countDistinct("distinct1").alias("distinct1"))
#dd.explain("formatted")

In [120]:
dd.explain("formatted")

== Physical Plan ==
AdaptiveSparkPlan (14)
+- HashAggregate (13)
   +- Exchange (12)
      +- HashAggregate (11)
         +- HashAggregate (10)
            +- Exchange (9)
               +- HashAggregate (8)
                  +- HashAggregate (7)
                     +- Exchange (6)
                        +- HashAggregate (5)
                           +- HashAggregate (4)
                              +- Exchange (3)
                                 +- HashAggregate (2)
                                    +- Scan parquet  (1)


(1) Scan parquet 
Output [3]: [user_id#117, sales_amount#118, date#119]
Batched: true
Location: InMemoryFileIndex [file:/home/erdemb/libs/sparktacle/example_data]
ReadSchema: struct<user_id:int,sales_amount:int,date:int>

(2) HashAggregate
Input [3]: [user_id#117, sales_amount#118, date#119]
Keys [2]: [user_id#117, date#119]
Functions [1]: [partial_sum(sales_amount#118)]
Aggregate Attributes [1]: [sum(sales_amount#118)#128L]
Results [3]: [user_id#117, date#119

In [5]:
df = spark.read.load("example_data")
dd = df.agg(F.sum("sales_amount"), F.countDistinct("date"))
dd.explain("formatted")

== Physical Plan ==
AdaptiveSparkPlan (8)
+- HashAggregate (7)
   +- Exchange (6)
      +- HashAggregate (5)
         +- HashAggregate (4)
            +- Exchange (3)
               +- HashAggregate (2)
                  +- Scan parquet  (1)


(1) Scan parquet 
Output [2]: [sales_amount#36, date#37]
Batched: true
Location: InMemoryFileIndex [file:/home/erdemb/libs/sparktacle/example_data]
ReadSchema: struct<sales_amount:int,date:int>

(2) HashAggregate
Input [2]: [sales_amount#36, date#37]
Keys [1]: [date#37]
Functions [1]: [partial_sum(sales_amount#36)]
Aggregate Attributes [1]: [sum(sales_amount#36)#42L]
Results [2]: [date#37, sum#54L]

(3) Exchange
Input [2]: [date#37, sum#54L]
Arguments: hashpartitioning(date#37, 200), ENSURE_REQUIREMENTS, [plan_id=17]

(4) HashAggregate
Input [2]: [date#37, sum#54L]
Keys [1]: [date#37]
Functions [1]: [merge_sum(sales_amount#36)]
Aggregate Attributes [1]: [sum(sales_amount#36)#42L]
Results [2]: [date#37, sum#54L]

(5) HashAggregate
Input [2]: [date

In [127]:
gateway_client = spark._jvm.org.apache.spark.api.java.JavaSparkContext(spark._jsc).gateway().gateway_client



Py4JError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext. Trace:
py4j.Py4JException: Constructor org.apache.spark.api.java.JavaSparkContext([class org.apache.spark.api.java.JavaSparkContext]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:180)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:197)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.lang.Thread.run(Thread.java:750)



In [126]:
from py4j.java_gateway import JavaObject

logical_plan = dd._jdf.queryExecution().logical()

# Create a StringBuilder object to store the plan
gateway_client = spark.sparkContext._gateway.java_gateway

# Obtain the logical plan as a Java object
logical_plan = df._jdf.queryExecution().logical()

# Create a StringBuilder object to store the plan
string_builder = JavaObject("java.lang.StringBuilder", gateway_client)()

# Use explain() with logical=True and provide the StringBuilder object
logical_plan.explain(True, string_builder)


AttributeError: 'JavaMember' object has no attribute 'gateway_property'

In [96]:
dd.explain("extended")

== Parsed Logical Plan ==
'Aggregate [sum('sales) AS sales#142, 'count(distinct 'distinct1) AS distinct1#143]
+- Project [user_id#117, sales#129L, date_distinct#130L, (cast(1 as bigint) + date_distinct#130L) AS distinct1#136L]
   +- Aggregate [user_id#117], [user_id#117, sum(sales_amount#118) AS sales#129L, count(distinct date#119) AS date_distinct#130L]
      +- Relation [id#116L,user_id#117,sales_amount#118,date#119] parquet

== Analyzed Logical Plan ==
sales: bigint, distinct1: bigint
Aggregate [sum(sales#129L) AS sales#142L, count(distinct distinct1#136L) AS distinct1#143L]
+- Project [user_id#117, sales#129L, date_distinct#130L, (cast(1 as bigint) + date_distinct#130L) AS distinct1#136L]
   +- Aggregate [user_id#117], [user_id#117, sum(sales_amount#118) AS sales#129L, count(distinct date#119) AS date_distinct#130L]
      +- Relation [id#116L,user_id#117,sales_amount#118,date#119] parquet

== Optimized Logical Plan ==
Aggregate [sum(sales#129L) AS sales#142L, count(distinct distinc

In [143]:
logical_plan = dd._jdf.queryExecution().logical()


In [119]:
print(logical_plan.prettyJson())

[ {
  "class" : "org.apache.spark.sql.catalyst.plans.logical.Aggregate",
  "num-children" : 1,
  "groupingExpressions" : [ ],
  "aggregateExpressions" : [ [ {
    "class" : "org.apache.spark.sql.catalyst.expressions.Alias",
    "num-children" : 1,
    "child" : 0,
    "name" : "sales",
    "exprId" : {
      "product-class" : "org.apache.spark.sql.catalyst.expressions.ExprId",
      "id" : 142,
      "jvmId" : "56114ab7-d7ee-4182-855b-6e8c061b6b7c"
    },
    "qualifier" : [ ],
    "nonInheritableMetadataKeys" : "[__dataset_id, __col_position]"
  }, {
    "class" : "org.apache.spark.sql.catalyst.expressions.aggregate.AggregateExpression",
    "num-children" : 1,
    "aggregateFunction" : 0,
    "mode" : {
      "object" : "org.apache.spark.sql.catalyst.expressions.aggregate.Complete$"
    },
    "isDistinct" : false,
    "resultId" : {
      "product-class" : "org.apache.spark.sql.catalyst.expressions.ExprId",
      "id" : 141,
      "jvmId" : "56114ab7-d7ee-4182-855b-6e8c061b6b7c"
   

In [100]:
dir(logical_plan)

['$anonfun$_isStreaming$1',
 '$anonfun$_isStreaming$1$adapted',
 '$anonfun$allAttributes$1',
 '$anonfun$argString$1',
 '$anonfun$asCode$1',
 '$anonfun$canonicalized$1',
 '$anonfun$childAttributes$1',
 '$anonfun$childMetadataAttributes$1',
 '$anonfun$childrenResolved$1',
 '$anonfun$childrenResolved$1$adapted',
 '$anonfun$clone$1',
 '$anonfun$clone$2',
 '$anonfun$clone$3',
 '$anonfun$clone$4',
 '$anonfun$collect$1',
 '$anonfun$collect$1$adapted',
 '$anonfun$collect$2',
 '$anonfun$collectFirst$1',
 '$anonfun$collectFirst$2',
 '$anonfun$collectFirst$3',
 '$anonfun$collectWithSubqueries$1',
 '$anonfun$deterministic$1',
 '$anonfun$deterministic$1$adapted',
 '$anonfun$deterministic$2',
 '$anonfun$deterministic$2$adapted',
 '$anonfun$doCanonicalize$1',
 '$anonfun$doCanonicalize$2',
 '$anonfun$exists$1',
 '$anonfun$exists$1$adapted',
 '$anonfun$expressions$1',
 '$anonfun$expressions$2',
 '$anonfun$find$1',
 '$anonfun$find$2',
 '$anonfun$flatMap$1',
 '$anonfun$flatMap$1$adapted',
 '$anonfun$fore

In [149]:
print(logical_plan.treeString())

'Aggregate [sum('sales) AS sales#142, 'count(distinct 'distinct1) AS distinct1#143]
+- Project [user_id#117, sales#129L, date_distinct#130L, (cast(1 as bigint) + date_distinct#130L) AS distinct1#136L]
   +- Aggregate [user_id#117], [user_id#117, sum(sales_amount#118) AS sales#129L, count(distinct date#119) AS date_distinct#130L]
      +- Relation [id#116L,user_id#117,sales_amount#118,date#119] parquet



In [137]:
logical_plan.

TypeError: 'JavaObject' object is not callable

In [6]:
dd.explain("cost")

== Optimized Logical Plan ==
Aggregate [sum(sales_amount#88) AS sum(sales_amount)#99L, count(distinct date#89) AS count(date)#100L], Statistics(sizeInBytes=24.0 B, rowCount=1)
+- Project [sales_amount#88, date#89], Statistics(sizeInBytes=3.1 KiB)
   +- Relation [id#86L,user_id#87,sales_amount#88,date#89] parquet, Statistics(sizeInBytes=5.3 KiB)

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[sum(sales_amount#88), count(distinct date#89)], output=[sum(sales_amount)#99L, count(date)#100L])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#95]
      +- HashAggregate(keys=[], functions=[merge_sum(sales_amount#88), partial_count(distinct date#89)], output=[sum#105L, count#108L])
         +- HashAggregate(keys=[date#89], functions=[merge_sum(sales_amount#88)], output=[date#89, sum#105L])
            +- Exchange hashpartitioning(date#89, 200), ENSURE_REQUIREMENTS, [id=#91]
               +- HashAggregate(keys=[date#89], functions=[partial_

In [ ]:
dd.explain("formatted")

In [5]:
dd.explain("extended")

== Parsed Logical Plan ==
'Aggregate ['user_id], ['user_id, sum('sales_amount) AS sum(sales_amount)#31, 'count(distinct 'date) AS count(date)#32]
+- Relation [id#18L,user_id#19,sales_amount#20,date#21] parquet

== Analyzed Logical Plan ==
user_id: int, sum(sales_amount): bigint, count(date): bigint
Aggregate [user_id#19], [user_id#19, sum(sales_amount#20) AS sum(sales_amount)#31L, count(distinct date#21) AS count(date)#32L]
+- Relation [id#18L,user_id#19,sales_amount#20,date#21] parquet

== Optimized Logical Plan ==
Aggregate [user_id#19], [user_id#19, sum(sales_amount#20) AS sum(sales_amount)#31L, count(distinct date#21) AS count(date)#32L]
+- Project [user_id#19, sales_amount#20, date#21]
   +- Relation [id#18L,user_id#19,sales_amount#20,date#21] parquet

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[user_id#19], functions=[sum(sales_amount#20), count(distinct date#21)], output=[user_id#19, sum(sales_amount)#31L, count(date)#32L])
   +- Exchange hashp

NoneType

In [51]:
import io
from contextlib import redirect_stdout

f = io.StringIO()
with redirect_stdout(f):
    dd.explain("extended")
out = f.getvalue()


In [34]:
out_list

['',
 'Parsed Logical Plan',
 "'Aggregate [sum('sales_amount) AS sum(sales_amount)#56, 'count(distinct 'date) AS count(date)#57]\n+- Relation [id#43L,user_id#44,sales_amount#45,date#46] parquet",
 'Analyzed Logical Plan',
 'sum(sales_amount): bigint, count(date): bigint\nAggregate [sum(sales_amount#45) AS sum(sales_amount)#56L, count(distinct date#46) AS count(date)#57L]\n+- Relation [id#43L,user_id#44,sales_amount#45,date#46] parquet',
 'Optimized Logical Plan',
 'Aggregate [sum(sales_amount#45) AS sum(sales_amount)#56L, count(distinct date#46) AS count(date)#57L]\n+- Project [sales_amount#45, date#46]\n   +- Relation [id#43L,user_id#44,sales_amount#45,date#46] parquet',
 'Physical Plan',
 'AdaptiveSparkPlan isFinalPlan=false\n+- HashAggregate(keys=[], functions=[sum(sales_amount#45), count(distinct date#46)], output=[sum(sales_amount)#56L, count(date)#57L])\n   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#58]\n      +- HashAggregate(keys=[], functions=[merge_sum(sales_amoun

In [48]:
def get_plan(df, mode="extended"):
    f = io.StringIO()
    with redirect_stdout(f):
        df.explain(mode)
    out = f.getvalue()

    return out

In [52]:
formatted = get_plan(dd, "formatted")

In [54]:
print(formatted)

== Physical Plan ==
AdaptiveSparkPlan (8)
+- HashAggregate (7)
   +- Exchange (6)
      +- HashAggregate (5)
         +- HashAggregate (4)
            +- Exchange (3)
               +- HashAggregate (2)
                  +- Scan parquet  (1)


(1) Scan parquet 
Output [3]: [user_id#63, sales_amount#64, date#65]
Batched: true
Location: InMemoryFileIndex [file:/home/erdemb/libs/sparktacle/example_data]
ReadSchema: struct<user_id:int,sales_amount:int,date:int>

(2) HashAggregate
Input [3]: [user_id#63, sales_amount#64, date#65]
Keys [2]: [user_id#63, date#65]
Functions [1]: [partial_sum(sales_amount#64)]
Aggregate Attributes [1]: [sum(sales_amount#64)#74L]
Results [3]: [user_id#63, date#65, sum#83L]

(3) Exchange
Input [3]: [user_id#63, date#65, sum#83L]
Arguments: hashpartitioning(user_id#63, date#65, 200), ENSURE_REQUIREMENTS, [plan_id=49]

(4) HashAggregate
Input [3]: [user_id#63, date#65, sum#83L]
Keys [2]: [user_id#63, date#65]
Functions [1]: [merge_sum(sales_amount#64)]
Aggregate At

In [105]:
all_operations = [x.strip() for x in formatted.split("\n\n")]
all_operations = [x for x in all_operations if len(x) > 0]
all_operations = all_operations[1:]

In [108]:
operation_names = [x.split("\n")[0] for x in all_operations]

In [113]:
scan_operations = [x for x in operation_names if "Scan" in x]
scan_indexes = [operation_names.index(x) for x in scan_operations]

In [116]:
scan_string = all_operations[0]

In [130]:
columns = [x for x in scan_string.split("\n") if x.startswith("Output")][0]
path = [x for x in scan_string.split("\n") if x.startswith("Location")][0]

In [133]:
columns_text = re.findall(r'\[(.*?)\]', columns)[-1]

In [136]:
path_text = re.findall(r'\[(.*?)\]', path)[-1]

['file:/home/erdemb/libs/sparktacle/example_data']

In [128]:
import re

output = "Output [2]: [sales_amount#45, date#46]"
matches = re.findall(r'\[(.*?)\]', output)
result = matches[-1]
result

'sales_amount#45, date#46'

In [98]:
scan_string.split("\n")

['(1) Scan parquet ',
 'Output [2]: [sales_amount#45, date#46]',
 'Batched: true',
 'Location: InMemoryFileIndex [file:/home/erdemb/libs/sparktacle/example_data]',
 'ReadSchema: struct<sales_amount:int,date:int>']

In [36]:
out = get_plan(dd)
out_list = [x.strip() for x in out.split("==")]
out_list = [x for x in out_list if len(x) > 0]


In [45]:
available_plans = []
for x in range(0, len(out_list), 2):
    available_plans.append(out_list[x])

In [54]:
physical_lines = [x.strip() for x in out_list[7].split("+-")]

In [56]:
from_source = [x for x in physical_lines if x.startswith('FileScan')]

In [ ]:
# Columns [sales_amount#45,date#46], path: /home/erdemb/libs/sparktacle/example_data

In [69]:
from_source

['FileScan parquet [sales_amount#45,date#46] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/home/erdemb/libs/sparktacle/example_data], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<sales_amount:int,date:int>']

In [68]:
file_paths = [x for x in from_source[0].split() if "file:" in x]
file_paths[0].split("file:")[1].split("]")[0]

'/home/erdemb/libs/sparktacle/example_data'

In [15]:
out_list.index("Parsed Logical Plan")
plan = out_list[2]

In [19]:
plan = [x.strip() for x in plan.split("+-")]

In [20]:
plan.e

["'Aggregate ['user_id], ['user_id, sum('sales_amount) AS sum(sales_amount)#31, 'count(distinct 'date) AS count(date)#32]",
 'Relation [id#18L,user_id#19,sales_amount#20,date#21] parquet']

In [9]:
dd = df.groupBy("user_id").agg(F.sum("sales_amount"), F.countDistinct("date"))
dd.explain("extended")

== Parsed Logical Plan ==
'Aggregate ['user_id], ['user_id, sum('sales_amount) AS sum(sales_amount)#115, 'count(distinct 'date) AS count(date)#116]
+- Relation [id#86L,user_id#87,sales_amount#88,date#89] parquet

== Analyzed Logical Plan ==
user_id: int, sum(sales_amount): bigint, count(date): bigint
Aggregate [user_id#87], [user_id#87, sum(sales_amount#88) AS sum(sales_amount)#115L, count(distinct date#89) AS count(date)#116L]
+- Relation [id#86L,user_id#87,sales_amount#88,date#89] parquet

== Optimized Logical Plan ==
Aggregate [user_id#87], [user_id#87, sum(sales_amount#88) AS sum(sales_amount)#115L, count(distinct date#89) AS count(date)#116L]
+- Project [user_id#87, sales_amount#88, date#89]
   +- Relation [id#86L,user_id#87,sales_amount#88,date#89] parquet

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[user_id#87], functions=[sum(sales_amount#88), count(distinct date#89)], output=[user_id#87, sum(sales_amount)#115L, count(date)#116L])
   +- Exchan

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.sql_formatter import SQLStringFormatter

spark = SparkSession.builder.getOrCreate()



In [3]:
query_execution = dd._jdf.queryExecution()
optimized_plan = query_execution.optimizedPlan()


In [40]:
spark_plan = dd._jdf.queryExecution().executedPlan().prettyJson()

In [45]:
dd.explain(mode="codegen")

Found 0 WholeStageCodegen subtrees.



In [24]:
spark._jvm.org.apache.spark.sql.execution.SparkSqlParser().parsePlan(spark_plan).toSQL()

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near '['.(line 1, pos 0)

== SQL ==
[ {
^^^
  "class" : "org.apache.spark.sql.execution.adaptive.AdaptiveSparkPlanExec",
  "num-children" : 0,
  "inputPlan" : [ {
    "class" : "org.apache.spark.sql.execution.aggregate.HashAggregateExec",
    "num-children" : 1,
    "requiredChildDistributionExpressions" : [ [ {
      "class" : "org.apache.spark.sql.catalyst.expressions.AttributeReference",
      "num-children" : 0,
      "name" : "user_id",
      "dataType" : "integer",
      "nullable" : true,
      "metadata" : { },
      "exprId" : {
        "product-class" : "org.apache.spark.sql.catalyst.expressions.ExprId",
        "id" : 15,
        "jvmId" : "4a0824ab-ad84-4b89-a2a9-bbb55258949b"
      },
      "qualifier" : [ ]
    } ] ],
    "isStreaming" : false,
    "groupingExpressions" : [ [ {
      "class" : "org.apache.spark.sql.catalyst.expressions.AttributeReference",
      "num-children" : 0,
      "name" : "user_id",
      "d

In [25]:
# Generate the Spark plan
spark_plan = dd._jdf.queryExecution().executedPlan().prettyJson()

# Replace temporary table references with physical table names
for table_name, physical_name in spark.catalog.listTables():
    if physical_name.startswith("spark_catalog"):
        spark_plan = spark_plan.replace(table_name, physical_name)

In [28]:
import json

In [29]:
spark_plan_json = json.dumps(spark_plan)

In [34]:
spark_plan_list = eval(spark_plan)

# Generate a SQL query for each dictionary in the list
for plan_dict in spark_plan_list:
    try:
        sql_query = spark._jvm.org.apache.spark.sql.execution.SparkSqlParser().parsePlan(str(plan_dict)).toSQL()
        print(sql_query)
    except AnalysisException as e:
        print("Failed to generate SQL query from Spark plan:", e)


NameError: name 'true' is not defined

In [36]:
sql_query = spark._jvm.org.apache.spark.sql.execution.SparkSqlParser().parsePlan(check).toSQL()


ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near '{'.(line 2, pos 1)

== SQL ==

 {
-^^^
  "class" : "org.apache.spark.sql.execution.adaptive.AdaptiveSparkPlanExec",
  "num-children" : 0,
  "inputPlan" : [ {
    "class" : "org.apache.spark.sql.execution.aggregate.HashAggregateExec",
    "num-children" : 1,
    "requiredChildDistributionExpressions" : [ [ {
      "class" : "org.apache.spark.sql.catalyst.expressions.AttributeReference",
      "num-children" : 0,
      "name" : "user_id",
      "dataType" : "integer",
      "nullable" : true,
      "metadata" : { },
      "exprId" : {
        "product-class" : "org.apache.spark.sql.catalyst.expressions.ExprId",
        "id" : 15,
        "jvmId" : "4a0824ab-ad84-4b89-a2a9-bbb55258949b"
      },
      "qualifier" : [ ]
    } ] ],
    "isStreaming" : false,
    "groupingExpressions" : [ [ {
      "class" : "org.apache.spark.sql.catalyst.expressions.AttributeReference",
      "num-children" : 0,
      "name" : "user_id",
      "

In [31]:
sql_query = spark._jvm.org.apache.spark.sql.execution.SparkSqlParser().parsePlan(spark_plan).toSQL()


ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near '"[ {\n  \"class\" : \"org.apache.spark.sql.execution.adaptive.AdaptiveSparkPlanExec\",\n  \"num-children\" : 0,\n  \"inputPlan\" : [ {\n    \"class\" : \"org.apache.spark.sql.execution.aggregate.HashAggregateExec\",\n    \"num-children\" : 1,\n    \"requiredChildDistributionExpressions\" : [ [ {\n      \"class\" : \"org.apache.spark.sql.catalyst.expressions.AttributeReference\",\n      \"num-children\" : 0,\n      \"name\" : \"user_id\",\n      \"dataType\" : \"integer\",\n      \"nullable\" : true,\n      \"metadata\" : { },\n      \"exprId\" : {\n        \"product-class\" : \"org.apache.spark.sql.catalyst.expressions.ExprId\",\n        \"id\" : 15,\n        \"jvmId\" : \"4a0824ab-ad84-4b89-a2a9-bbb55258949b\"\n      },\n      \"qualifier\" : [ ]\n    } ] ],\n    \"isStreaming\" : false,\n    \"groupingExpressions\" : [ [ {\n      \"class\" : \"org.apache.spark.sql.catalyst.expressions.AttributeReference\",\n      \"num-ch

In [43]:
jdf_plan = dd._jdf.logicalPlan().prettyJson()

In [44]:
jdf_plan_dict = json.loads(jdf_plan)

In [71]:
jdf_plan_dict[0]["groupingExpressions"]

[[{'class': 'org.apache.spark.sql.catalyst.expressions.AttributeReference',
   'num-children': 0,
   'name': 'user_id',
   'dataType': 'integer',
   'nullable': True,
   'metadata': {},
   'exprId': {'product-class': 'org.apache.spark.sql.catalyst.expressions.ExprId',
    'id': 60,
    'jvmId': '56114ab7-d7ee-4182-855b-6e8c061b6b7c'},
   'qualifier': []}]]

In [90]:
jdf_plan_dict[0]["aggregateExpressions"][2][0]

{'class': 'org.apache.spark.sql.catalyst.expressions.Alias',
 'num-children': 1,
 'child': 0,
 'name': 'count(date)',
 'exprId': {'product-class': 'org.apache.spark.sql.catalyst.expressions.ExprId',
  'id': 73,
  'jvmId': '56114ab7-d7ee-4182-855b-6e8c061b6b7c'},
 'qualifier': [],
 'nonInheritableMetadataKeys': []}

In [67]:
[x[0]["name"] for x in jdf_plan_dict[1]["output"]]

['id', 'user_id', 'sales_amount', 'date']

In [92]:
jdf_plan_dict[0]["aggregateExpressions"][2]

[{'class': 'org.apache.spark.sql.catalyst.expressions.Alias',
  'num-children': 1,
  'child': 0,
  'name': 'count(date)',
  'exprId': {'product-class': 'org.apache.spark.sql.catalyst.expressions.ExprId',
   'id': 73,
   'jvmId': '56114ab7-d7ee-4182-855b-6e8c061b6b7c'},
  'qualifier': [],
  'nonInheritableMetadataKeys': []},
 {'class': 'org.apache.spark.sql.catalyst.expressions.aggregate.AggregateExpression',
  'num-children': 1,
  'aggregateFunction': 0,
  'mode': {'object': 'org.apache.spark.sql.catalyst.expressions.aggregate.Complete$'},
  'isDistinct': True,
  'resultId': {'product-class': 'org.apache.spark.sql.catalyst.expressions.ExprId',
   'id': 74,
   'jvmId': '56114ab7-d7ee-4182-855b-6e8c061b6b7c'}},
 {'class': 'org.apache.spark.sql.catalyst.expressions.aggregate.Count',
  'num-children': 1,
  'children': [0]},
 {'class': 'org.apache.spark.sql.catalyst.expressions.AttributeReference',
  'num-children': 0,
  'name': 'date',
  'dataType': 'integer',
  'nullable': True,
  'metada

In [83]:
jdf_plan_dict[1].keys()

dict_keys(['class', 'num-children', 'relation', 'output', 'isStreaming'])

In [8]:
plan = dd._jdf.queryExecution().executedPlan()

In [31]:
import ujson as json

pretty_plan = plan.prettyJson()
pretty_plan = json.loads(pretty_plan)

In [42]:
pretty_plan[0]["inputPlan"]

[{'class': 'org.apache.spark.sql.execution.aggregate.HashAggregateExec',
  'num-children': 1,
  'requiredChildDistributionExpressions': [[{'class': 'org.apache.spark.sql.catalyst.expressions.AttributeReference',
     'num-children': 0,
     'name': 'user_id',
     'dataType': 'integer',
     'nullable': True,
     'metadata': {},
     'exprId': {'product-class': 'org.apache.spark.sql.catalyst.expressions.ExprId',
      'id': 60,
      'jvmId': '56114ab7-d7ee-4182-855b-6e8c061b6b7c'},
     'qualifier': []}]],
  'isStreaming': False,
  'groupingExpressions': [[{'class': 'org.apache.spark.sql.catalyst.expressions.AttributeReference',
     'num-children': 0,
     'name': 'user_id',
     'dataType': 'integer',
     'nullable': True,
     'metadata': {},
     'exprId': {'product-class': 'org.apache.spark.sql.catalyst.expressions.ExprId',
      'id': 60,
      'jvmId': '56114ab7-d7ee-4182-855b-6e8c061b6b7c'},
     'qualifier': []}]],
  'aggregateExpressions': [[{'class': 'org.apache.spark.sql

In [24]:
dir(plan)

['$anonfun$allAttributes$1',
 '$anonfun$argString$1',
 '$anonfun$asCode$1',
 '$anonfun$canonicalized$1',
 '$anonfun$cleanUpAndThrowException$1',
 '$anonfun$cleanUpAndThrowException$1$adapted',
 '$anonfun$cleanUpAndThrowException$2',
 '$anonfun$cleanUpAndThrowException$3',
 '$anonfun$cleanUpAndThrowException$4',
 '$anonfun$cleanUpAndThrowException$4$adapted',
 '$anonfun$cleanUpTempTags$1',
 '$anonfun$cleanUpTempTags$1$adapted',
 '$anonfun$cleanupResources$1',
 '$anonfun$cleanupResources$1$adapted',
 '$anonfun$clone$1',
 '$anonfun$clone$2',
 '$anonfun$clone$3',
 '$anonfun$clone$4',
 '$anonfun$collect$1',
 '$anonfun$collect$1$adapted',
 '$anonfun$collect$2',
 '$anonfun$collectFirst$1',
 '$anonfun$collectFirst$2',
 '$anonfun$collectFirst$3',
 '$anonfun$collectWithSubqueries$1',
 '$anonfun$createQueryStages$1',
 '$anonfun$createQueryStages$2',
 '$anonfun$createQueryStages$3',
 '$anonfun$createQueryStages$4',
 '$anonfun$createQueryStages$4$adapted',
 '$anonfun$createQueryStages$5',
 '$anonfu

In [15]:
dir(query_execution)

['$anonfun$analyzed$1',
 '$anonfun$commandExecuted$1',
 '$anonfun$executePhase$1',
 '$anonfun$executePhase$2',
 '$anonfun$executedPlan$1',
 '$anonfun$explainString$1',
 '$anonfun$explainString$1$adapted',
 '$anonfun$normalized$1',
 '$anonfun$optimizedPlan$1',
 '$anonfun$simpleString$1',
 '$anonfun$simpleString$1$adapted',
 '$anonfun$simpleString$2',
 '$anonfun$sparkPlan$1',
 '$anonfun$stringWithStats$1',
 '$anonfun$stringWithStats$1$adapted',
 '$anonfun$stringWithStats$3',
 '$anonfun$stringWithStats$4',
 '$anonfun$toString$1',
 '$anonfun$toString$1$adapted',
 '$anonfun$withCachedData$1',
 '$anonfun$writePlans$1',
 '$anonfun$writePlans$2',
 '$anonfun$writePlans$3',
 '$anonfun$writePlans$4',
 '$anonfun$writePlans$5',
 '$lessinit$greater$default$3',
 '$lessinit$greater$default$4',
 '_ipython_canary_method_should_not_exist_',
 'analyzed',
 'assertAnalyzed',
 'assertCommandExecuted',
 'assertSupported',
 'commandExecuted',
 'createSparkPlan',
 'debug',
 'equals',
 'executePhase',
 'executed

In [22]:
print(query_execution.toString())

== Parsed Logical Plan ==
'Aggregate ['user_id], ['user_id, sum('sales_amount) AS sum(sales_amount)#72, 'count(distinct 'date) AS count(date)#73]
+- Relation [id#59L,user_id#60,sales_amount#61,date#62] parquet

== Analyzed Logical Plan ==
user_id: int, sum(sales_amount): bigint, count(date): bigint
Aggregate [user_id#60], [user_id#60, sum(sales_amount#61) AS sum(sales_amount)#72L, count(distinct date#62) AS count(date)#73L]
+- Relation [id#59L,user_id#60,sales_amount#61,date#62] parquet

== Optimized Logical Plan ==
Aggregate [user_id#60], [user_id#60, sum(sales_amount#61) AS sum(sales_amount)#72L, count(distinct date#62) AS count(date)#73L]
+- Project [user_id#60, sales_amount#61, date#62]
   +- Relation [id#59L,user_id#60,sales_amount#61,date#62] parquet

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[user_id#60], functions=[sum(sales_amount#61), count(distinct date#62)], output=[user_id#60, sum(sales_amount)#72L, count(date)#73L])
   +- Exchange hashp

In [13]:
query_execution = dd._jdf.queryExecution()

# Get the execution plan as a string
explain_output = query_execution.toString()

# Convert the execution plan to JSON
execution_plan_json = query_execution.toJSON().first()



Py4JError: An error occurred while calling o79.toJSON. Trace:
py4j.Py4JException: Method toJSON([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:321)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:329)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.lang.Thread.run(Thread.java:750)



In [9]:
from pyspark._jvm import JClass

ModuleNotFoundError: No module named 'pyspark._jvm'

In [11]:
spark._jvm.execution.SparkPlanGraph(plan)

TypeError: 'JavaPackage' object is not callable

In [ ]:
# Groupby, filter, join, union, window